# Prompt Engineering Text-to-QueryDSL Using Claude 3 Models 

---
## Introduction

In this notebook we step through how to leverage Claude 3 models to create a Text-to-queryDSL. Also we step through how to generate synthetic data for faster testing and development for this example using Claude 3 model. OpenSearch (as well as Elasticsearch) provide a search language called query domain-specific language (DSL) that you can use to search your data. Query DSL is a flexible language with a JSON interface. With query DSL, you need to specify a query in the query parameter of the search.

The use case for this notebook is a health social media app. The health app includes different posts on the platform for users as well as user profiles for the health application.

This solution is intended to help teams looking for Text-to-queryDSL use cases as well as how to develop test data around existing schemas to test functionality. All the code is intended for development purposes only.

--- 
## Anthropic Claude 3 Model Selection
There are multiple different models available on Amazon Bedrock from Anthropic but the main models covered in this notebook come from the Anthropic Claude 3 family:

### 1. Claude 3.5 Sonnet
- **Description:** Anthropic’s most intelligent and advanced model, Claude 3.5 Sonnet, demonstrates exceptional capabilities across a diverse range of tasks and evaluations while also outperforming Claude 3 Opus.
- **MaxTokens:** 200K
- **Context Window:** 200K
- **Languages:** English, Spanish, Japanese, and multiple other languages.
- **Supported Use cases:**  RAG or search & retrieval over vast amounts of knowledge, product recommendations, forecasting, targeted marketing, code generation, quality control, parse text from images.

### 2. Claude 3 Sonnet
- **Description:** Claude 3 Sonnet is engineered to be dependable for scaled AI deployments across a variety of use cases.
- **MaxTokens:** 200K
- **Context Window:** 200K
- **Languages:** English, Spanish, Japanese, and multiple other languages.
- **Supported Use cases:** RAG or search & retrieval over vast amounts of knowledge, product recommendations, forecasting, targeted marketing, code generation, quality control, parse text from images.

### 3. Claude 3 Haiku
- **Description:** Anthropic’s fastest, most compact model for near-instant responsiveness. It answers simple queries and requests with speed.
- **MaxTokens:** 200K
- **Context Window:** 200K
- **Languages:** English, Spanish, Japanese, and multiple other languages.
- **Supported Use cases:** Quick and accurate support in live interactions, translations, content moderation, optimize logistics, inventory management, extract knowledge from unstructured data.

We chose from the Claude 3 model family for this workload to compare performance, accuracy and cost. Haiku is faster than Sonnet when running inference on Amazon Bedrock; however, Claude 3 Sonnet and Sonnet 3.5 have improved sustained accuracy.

--- 

## The Approach to text-to-QueryDSL for Development Teams

When developing solutions from the ground up, it can be difficult to test when there is no example data present as well as prompting Claude models for the intended tasks. 

Thus, in this notebook we walk through how to develop synthetic schemas, synthetic data based on the schemas, example questions to ask the synthetic data and then running the query against a Opensearch Serverless collection to understand if the generated query is extracting relevant data from the collection. In this example we are using an Opensearch Serverless Collection as opposed to showing Elasticsearch cluster as many customers have migrated to more modern architecture/services. The approach is similar if you have an existing Elasticsearch cluster.

### Few-shot text-to-queryDSL prompting
For both syntehtic data creation as well as creating the query DSL based on text, the notebook demonstrates using the few-shot prompting approach and using system prompts. Overall when working with Claude models and building prompts for these models, Anthropic states the golden rule of clear prompting is "sow your prompt to a colleague, ideally someone who has minimal context on the task, and ask them to follow the instructions. If they’re confused, Claude will likely be too."

We take this same approach and ensure we are sending specific tasks to the LLM and specific examples to our system prompt to direct the model for each task at hand. For this use case, we start with two schemas "healthpost_schema" and "userprofile_schema" that our example social media health app is based on. Then we leverage these schemas to create the syntehtic data. We begin with these schemas in the proper formatting to create an index in an Opensearch Cluster. We use the synthetic data to create prompting examples to then pass to the LLM for few-shot prompting. Finally, we ingest this data into two indexes for the Opensearch Collection. To generate the Text-to-queryDSL query, we use few-shot prompting to pass in the examples to give context to the LLM of how we want the syntax of the query to look. 

The entire flow of this notebook also demonstrates prompt chaining as we are passing in one response from the LLM as the input to another request to the LLM.

---
## Prerequisites
1. Use kernel either conda_python3, conda_pytorch_p310 or conda_tensorflow2_p310.
2. Install the required packages.
3. Access to the Claude3 models on Amazon Bedrock and access to the Converse API.

---

## Getting Started
### Step 0: Install Dependencies and Import Modules

In [1]:
!pip install -U awscli -qU --force --quiet --no-warn-conflicts
!pip install boto3==1.34.127 -qU --force --quiet --no-warn-conflicts
!pip install numpy==1.26.4 -qU --force --quiet --no-warn-conflicts
!pip install opensearch-py -qU --force --quiet --no-warn-conflicts
!pip install requests-aws4auth -qU --force --quiet --no-warn-conflicts

Note: When installing libraries using the pip, you may encounter errors or warnings during the installation process. These are generally not critical and can be safely ignored. However, after installing the libraries, it is recommended to restart the kernel or computing environment you are working in. Restarting the kernel ensures that the newly installed libraries are loaded properly and available for use in your code or workflow.


In [2]:
import boto3
from botocore.exceptions import ClientError
import json
import os
from opensearchpy import OpenSearch, RequestsHttpConnection
from opensearchpy.helpers import bulk
import sagemaker
import time
import random
import re
from requests_aws4auth import AWS4Auth

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
# Setup Bedrock Client
bedrock_rt= boto3.client(
    service_name='bedrock-runtime'
)
session = boto3.session.Session()
region_name = session.region_name
# Create a SageMaker session that will be used when creatin the Opensearch Collection
sagemaker_role_arn = sagemaker.get_execution_role()
sagemaker_role_arn

'arn:aws:iam::709425451936:role/service-role/AmazonSageMaker-ExecutionRole-20240712T100744'

### Step 1: Read in the userprofile and healthpost schemas and corresponding index files

In [4]:
#Read in the userprofile and healthpost schema that are preset that are predefined
with open('schemas/userprofile_schema.json', 'r') as file:
    userprofile_schema = json.load(file)
#open the health posts scheams
with open('schemas/healthpost_schema.json', 'r') as file:
    healthpost_schema= json.load(file)

We also have a schemas formatted for the Opensearch index created later

In [5]:
with open('schemas/userprofile_schema_index.json', 'r') as file:
    userprofile_schema_index = json.load(file)
with open('schemas/healthpost_schema_index.json', 'r') as file:
    healthpost_schema_index= json.load(file)

We will need to access all the above information as strings later in the notebook so let's convert the json to strings for later use in the prompt

In [6]:
#get the correct formatting to send to system prompt
userprofile_schema_string= json.dumps(userprofile_schema, indent=2)
healthpost_schema_string= json.dumps(healthpost_schema, indent=2)
userprofile_index_string= json.dumps(userprofile_schema_index, indent=2)
healthpost_index_string= json.dumps(healthpost_schema_index, indent=2)

### Step 2:  Synthetic Data Generation Based on Schemas
Now since we have two schemas for healthposts and userprofiles for the health app, let's learn how to generate synthetic data for the schemas using a foundational model. We will use the generate_data() function throughout out the notebook as the single point of calling the LLM of choice

In [7]:
def generate_data(bedrock_rt, model_id, system_prompt, query, inference_config):
    """
    Function to call the Bedrock Converse API
    """
    messages = []
    messages.append({
        "role": "user",
        "content": [{"text": query}]
        }
    )
    response = bedrock_rt.converse(
        modelId=model_id,
        system=system_prompt,
        messages=messages,
        inferenceConfig=inference_config
    )
    output = response['output']['message']['content'][0]['text']
    return output

In [8]:
#Set up variables to switch the model_id depending on which model you'd like to test. We will start with sonnet 3.5 as default.
model_id_sonnet3_5 = "anthropic.claude-3-5-sonnet-20240620-v1:0"
model_id_sonnet = "anthropic.claude-3-sonnet-20240229-v1:0"
model_id_haiku = "anthropic.claude-3-haiku-20240307-v1:0"
model_id = model_id_sonnet
#we keep it at 0 because we don't want that much creativity
inference_config = {"temperature": 0}

We will set up two different messages to pass to the Converse API.
1. For health posts 
2. For user profiles. 

We separate the user prompt but keep the same system prompt for both calls. This is standard best practices as the user prompt can be subjective to different tasks at hand but system prompt will stay the same for both outputs. We also include a prompt to the LLM to leverage <json> html tags. This will help us later on when we need to convert the string output from the LLM into a readable JSON/dictionary for later use.

In [9]:
message_healthpost= f"""Please generate 5 Health Post entries based on the following JSON schema. Health Post Schema:{healthpost_schema_string}.
        Please provide the generated data in JSON format, do not include any other information in response other than the JSON outputs.
        Put each of these additional dictionaries in separate <json> tags."""

message_userprofile= f"""Please generate 5 user profile entries based on the following JSON schema. User Profile Schema: {healthpost_schema_string=}
        Please provide the generated data in JSON format, do not include any other information in response other than the JSON outputs
         Put each of these additional dictionaries in separate <json> tags"""

In [10]:
system_prompt = [{"text": """You are an AI assistant tasked with generating synthetic data for a health tech social media platform. You will be provided a JSON schema. Your job is to create realistic, diverse, and consistent sample data entries based on the schema.

Follow these guidelines:
1. Generate data that adheres strictly to the provided schemas.
2. Create diverse and realistic entries, considering various demographics, health conditions, and interests.
3. Ensure consistency between User Profile and Health Post data (e.g., usernames, user IDs).
4. Use realistic values for all fields, including dates, metrics, and engagement statistics.
5. Generate geolocation data for major cities around the world.
6. Create a mix of verified and non-verified users/posts.
7. Vary the sentiment scores and engagement metrics realistically.
8. Include a range of health interests, fitness levels, and medical conditions.
9. Generate realistic content for post titles and content fields.

Remember to maintain data privacy by not using real people's information. All data should be fictional but plausible.

"""
}
]

In [11]:
#Call the generate_data() function and store in a variable for now. You can loop throguh a few examples of this leveraging the same system prompt
health_post_data = generate_data(bedrock_rt, model_id_sonnet, system_prompt, message_healthpost, inference_config)
user_profile_data = generate_data(bedrock_rt, model_id_sonnet, system_prompt, message_userprofile, inference_config)

Claude doesn't have a formal "JSON Mode" with constrained sampling. We can still get reliable JSON from Claude and the code below is pulled from Anthropic's cookbook on how to extract the json. Above in our message we included an xml tag for <json> that allows us to denote when there is a separate output from the LLM and where we woudl need to reformat the response to be in json formatting

In [12]:
#this function extracts the string object between json tags
def extract_json_from_xml(input_string):
    # Regular expression to find content inside <json> tags
    match = re.search(r'<json>(.*?)</json>', input_string, re.DOTALL)
    return match.group(1) if match else None

In [13]:
#this function converts into a dictionary
def extract_between_tags(tag: str, string: str, strip: bool = False) -> list[str]:
    ext_list = re.findall(f"<{tag}>(.+?)</{tag}>", string, re.DOTALL)
    if strip:
        ext_list = [e.strip() for e in ext_list]
    return ext_list

health_posts_dict = [
    json.loads(d)
    for d in extract_between_tags("json", health_post_data)
]

user_profile_dict = [
    json.loads(d)
    for d in extract_between_tags("json", user_profile_data)
]

We will store our synthetic data in the 'data' folder, let's create it and then add the data to individual files for later use

In [14]:
if not os.path.exists('data'):
    os.makedirs('data')

In [15]:
if not os.path.exists('data/healthpost_data.json'):  
    with open('data/healthpost_data.json', 'w', encoding='utf-8') as f:
        json.dump(health_posts_dict, f, ensure_ascii=False, indent=4)
if not os.path.exists('data/userprofile_data.json'):  
    with open('data/userprofile_data.json', 'w', encoding='utf-8') as f:
        json.dump(user_profile_dict, f, ensure_ascii=False, indent=4)

Now we have our raw data contained in "health_posts_dict" and "user_profile_dict" variables. Let's move onto the next step to create an Opensearch Collection, Opensearch Index and ingest the data.

### Step 3: Create Opensearch Collection and Index

In [16]:
#create initial client for opensearch
aoss_client = boto3.client('opensearchserverless')
suffix = random.randrange(200, 900)
identity = boto3.client('sts').get_caller_identity()['Arn']

Find code to step through creating Opensearch Collections. Code sampled from here: https://github.com/aws-samples/Cohere-on-AWS/blob/main/cohere-cookbooks/Embeddings/Cohere_Embeddings_Search.ipynb

In [17]:
def create_policies_in_oss(es_name, aoss_client, role_arn):
    
    encryption_policy_name = f"sample-sp-{suffix}"
    network_policy_name = f"sample-np-{suffix}"
    access_policy_name = f'sample-ap-{suffix}'

    try:
        encryption_policy = aoss_client.create_security_policy(
            name=encryption_policy_name,
            policy=json.dumps(
                {
                    'Rules': [{'Resource': ['collection/' + es_name],
                               'ResourceType': 'collection'}],
                    'AWSOwnedKey': True
                }),
            type='encryption'
        )
    except Exception as ex:
        print(ex)
    
    try:
        network_policy = aoss_client.create_security_policy(
            name=network_policy_name,
            policy=json.dumps(
                [
                    {'Rules': [{'Resource': ['collection/' + es_name],
                                'ResourceType': 'collection'}],
                     'AllowFromPublic': True}
                ]),
            type='network'
        )
    except Exception as ex:
        print(ex)
    
    try:
        
        access_policy = aoss_client.create_access_policy(
            name=access_policy_name,
            policy=json.dumps(
                [
                    {
                        'Rules': [
                            {
                                'Resource': ['collection/' + es_name],
                                'Permission': [
                                    'aoss:CreateCollectionItems',
                                    'aoss:DeleteCollectionItems',
                                    'aoss:UpdateCollectionItems',
                                    'aoss:DescribeCollectionItems'],
                                'ResourceType': 'collection'
                            },
                            {
                                'Resource': ['index/' + es_name + '/*'],
                                'Permission': [
                                    'aoss:CreateIndex',
                                    'aoss:DeleteIndex',
                                    'aoss:UpdateIndex',
                                    'aoss:DescribeIndex',
                                    'aoss:ReadDocument',
                                    'aoss:WriteDocument'],
                                'ResourceType': 'index'
                            }],
                        'Principal': [identity, role_arn],
                        'Description': 'Easy data policy'}
                ]),
            type='data'
        )
    except Exception as ex:
        print(ex)
        
    return encryption_policy, network_policy, access_policy

**note**: **Only run the next cell once. If you run it more than once, will error since the policies already exist**

In [18]:
# Create Collection
es_name = f'es-collection-{suffix}'

encryption_policy, network_policy, access_policy = create_policies_in_oss(es_name=es_name,
                       aoss_client=aoss_client,
                       role_arn=sagemaker_role_arn)
#the type should be SEARCH, can be changed to VECTORSEARCH if we want a vectorDB
collection = aoss_client.create_collection(name=es_name,type='SEARCH')

**reminder**: only run the above cell ONCE

In [19]:
#extract the host from the Collection ID to be used 
collection_id = collection['createCollectionDetail']['id']
host = collection_id + '.' + region_name + '.aoss.amazonaws.com'
print(host)

l0pj31ls5ah6qj1cqht0.us-east-1.aoss.amazonaws.com


The following code will build the Opensearch client. 

In [20]:
service = 'aoss'
credentials= boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key,
                   region_name, service, session_token=credentials.token)
# Build the OpenSearch client
oss_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)
# It can take up to a minute for data access rules to be enforced
time.sleep(60)

**oss_client** is the variable denoted the Opensearch Collection. Now, let's create the two indices for our use case based on the data we read in earlier in the notebook

In [21]:
#health post
healthpost_index = "healthpost_index"
healthpost_body = {
   "settings": {
        "index": {
            "number_of_shards": 2,
            "number_of_replicas": 1
        }
    },
   "mappings": healthpost_schema_index['mappings']
}

#user profile
userprofile_index = "userprofile_index"
userprofile_body = {
   "settings": {
        "index": {
            "number_of_shards": 2,
            "number_of_replicas": 1
        }
    },
   "mappings": userprofile_schema_index['mappings']
}

Now, we ingest the data and let's check the response was received succesfully.
**note** sometimes it can take the collection anywhere from a few minutes to 10 minutes to create. If you are getting errors, wait a few more minutes or check status of your Opensearch Collection in AWS console. The status needs to be "active"

In [22]:
# We would get an index already exists exception if the index already exists, and that is okay. Ignore that error if it occurs
try:
    response_health = oss_client.indices.create(healthpost_index, body=healthpost_body) 
    response_user = oss_client.indices.create(userprofile_index, body=userprofile_body)
    print(f"response received for the create index -> {response_health}")
    print(f"response received for the create index -> {response_user}")

except Exception as e:
    print(f"error, exception={e}")

response received for the create index -> {'acknowledged': True, 'shards_acknowledged': True, 'index': 'healthpost_index'}
response received for the create index -> {'acknowledged': True, 'shards_acknowledged': True, 'index': 'userprofile_index'}


### Step 4: Ingest Synthetic Data into Opensearch

In [23]:
#read in the data created from earlier
with open('data/healthpost_data.json', 'r') as file:
    healthposts_json = json.load(file)
with open('data/userprofile_data.json', 'r') as file:
    userprofile_json = json.load(file)

In [24]:
def ingest_data(posts, user, healthpost_index, userprofile_index, oss_client):
    actions = []
    for k1, k2 in zip(healthposts_json, userprofile_json):
        actions.append(
        {
            "_index": healthpost_index,
            "_source": k1
        })
        actions.append(
        {
            "_index": userprofile_index,
            "_source": k2
        })
    success, failed = bulk(oss_client, actions)
    print(f"Successfully indexed {success} documents")
    print(f"Failed to index {len(failed)} documents")

In [25]:
ingest_data(posts= healthposts_json, user= userprofile_json, healthpost_index=healthpost_index, userprofile_index=userprofile_index, oss_client=oss_client)

Successfully indexed 10 documents
Failed to index 0 documents


If we get a succesful statement then we are good to go to the next step to create examples.

### Step 5: Create Examples for the Prompt using LLM and Generate the Query
Now, we will be setting up a user prompt and system prompt to pass into our generate_text() function to get the examples to user for the few-shot prompting

In [26]:
message_questions= """Generate 2 natural language questions and the corresponding natural language question based on 
provided data and scheams."""

In [27]:
system_prompt_query_generation = [
    {
        "text": f"""You are an expert query dsl generator. Your task is to take an input question and generate a query dsl to answer
        the question. Use the schemas and data below to generate the query. Put the query in json tags <json></json>
    Schemas:{userprofile_schema_string} {healthpost_schema_string}
    Data:{userprofile_json} {healthposts_json}
    
    Guidelines:
    - Ensure the generated query adheres to DSL query syntax
    - Do not created new mappings or other items that aren't included in the provided schemas.
    - Think through your answer before answering

    Output:
    - Only output the generated query with the corresponding generated question. The query should only be the raw query nothing else. Do not include any
    headers for the query"""
    }
]

In [28]:
system_prompt_questions= [{
    "text": f"""You are an expert query dsl generator. Your task is to provide a query dsl and the corresponding natural language question. Use 
    the schemas and data below to generate the query and question.   
    Schemas:{userprofile_schema_string} {healthpost_schema_string}
    Data:{userprofile_json} {healthposts_json}
    
    Guidelines:
    - Ensure the generated query adheres to DSL query syntax
    - Do not created new mappings or other items that aren't included in the provided schemas.
    - Think through your answer before answering

    Output:
    - Only output the generated query with the corresponding generated question. The query should only be the raw query nothing else. Do not include any
    headers for the query"""
}
]

In [29]:
example_prompt= generate_data(bedrock_rt, model_id_sonnet, system_prompt_query_generation, message_questions, inference_config)
print(example_prompt)

<json>
{
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "post_type": "recipe"
          }
        },
        {
          "range": {
            "likes_count": {
              "gte": 100
            }
          }
        },
        {
          "exists": {
            "field": "media_urls"
          }
        }
      ]
    }
  }
}
</json>

Question: Find all recipe posts that have at least 100 likes and include media URLs.

<json>
{
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "category": "fitness"
          }
        },
        {
          "range": {
            "health_metrics.steps": {
              "gte": 10000
            }
          }
        },
        {
          "range": {
            "health_metrics.heart_rate": {
              "gte": 150
            }
          }
        }
      ]
    }
  }
}
</json>

Question: Find all fitness-related posts where the user recorded at least 10,000 steps and had a h

Now, let's look at what the example looks like with the corresponding question and query

Now we are ready to configure the system prompt to help us create the es query. We will provide:
1. The schemas
2. The index names
3. The examples we've generated

We add "Guidelines" and "Output" fields to control the response of the LLM a bit more. 

In [30]:
# Define the system prompt for generating queries
system_prompt_query_generation = [
    {
        "text": f"""You are an expert query dsl generator. Your task is to take an input question and generate a query dsl to answer
        the question. Use the schemas and data below to generate the query. Put the query in json tags <json></json>
    Schemas:{userprofile_schema_string} {healthpost_schema_string}
    Data:{userprofile_json} {healthposts_json}
    
    Examples: {example_prompt}
    
    Guidelines:
    - Ensure the generated query adheres to DSL query syntax
    - Do not created new mappings or other items that aren't included in the provided schemas.
    - Think through your answer before answering

    Output:
    - Only output the generated query with the corresponding generated question. The query should only be the raw query nothing else. Do not include any
    headers for the query"""
    }
]

Let's pass in an example question to generate the query, then test it against our Opensearch Collection. Remember, we chose Claude 3 Sonnet model as default in the beginning. To test the other models, change out "model_id" when calling generate_data().

In [47]:
query ="What are some healthy recipes that include quinoa?"

In [48]:
# Generate responses and measure runtime with Haiku model
start_time = time.time()
query_response = generate_data(bedrock_rt, model_id, system_prompt_query_generation, query, inference_config)
print("Sonnet took", time.time() - start_time, "to run")
print("Response from Sonnet:")
print(query_response)

Sonnet took 7.273151397705078 to run
Response from Sonnet:
<json>
{
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "post_type": "recipe"
          }
        },
        {
          "match": {
            "content": "quinoa"
          }
        },
        {
          "match": {
            "tags": "healthy"
          }
        }
      ]
    }
  }
}
</json>


Define a function for running the generated query against the oss client:

In [49]:
def query_oss(query):
    #extract the json tags with the function generated beforehand
    dict1= [
    json.loads(d)
    for d in extract_between_tags("json", query)
    ]
    temp = dict1[0]
    response = oss_client.search(
    index = "_all",
    body= temp
    )
    return response

In [50]:
#query the oss client and evaluate results
query_oss(query_response)

{'took': 45,
 'timed_out': False,
 '_shards': {'total': 0, 'successful': 0, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 2.397612,
  'hits': [{'_index': 'userprofile_index',
    '_id': '7u-9m5EBiT38F8xt6mpC',
    '_score': 2.397612,
    '_source': {'user_id': 'user_1',
     'username': 'FitFoodie123',
     'post_type': 'recipe',
     'title': 'Healthy Quinoa Salad with Roasted Veggies',
     'content': 'Looking for a nutritious and delicious meal? Try this quinoa salad packed with roasted veggies like bell peppers, zucchini, and onions. Drizzle with a tangy lemon vinaigrette for an extra burst of flavor!',
     'created_at': '2023-05-01T12:34:56Z',
     'updated_at': '2023-05-01T12:34:56Z',
     'tags': ['healthy', 'vegetarian', 'meal-prep'],
     'category': 'food',
     'likes_count': 125,
     'comments_count': 32,
     'shares_count': 18,
     'media_urls': ['https://example.com/quinoa-salad.jpg'],
     'location': {'lat': 40.7128, '

Above we can see that the model was able to generate a query as well as accurately return data from the Opensearch Collection.

**note**: if you receive authorization errors eventually, just scroll up and rerun the cell that builds the Opensearch client. 

### Step 6: Generate Test Questions with LLM
In order to improve the amount of tests we have for this synthetic approach let's use the LLM to generate example questions we can start testing. This is helpful when evaluating the performance of the LLM and accuracy of responses.

In [36]:
#User prompt for generating the example questions
query_prompt= """Your task is to generate 5 example questions users can ask the health app based on provided schemas and data. Only include the questions generated
        in the response."""

In [37]:
# Define the system prompt for generating queries
query_system_prompt = [
    {
        "text": f"""Your task is to use the provided schemas and data to generate the questions. The questions you create, should be answered by the data provided. 
        Do not generate questions that cannot be answered by the data provided.
Schemas:
{userprofile_schema_string}
{healthpost_schema_string}

Data:{userprofile_json} {healthposts_json}

Guidelines:
- Create only the questions that can be answered by the provided information.
Output:
- Only output the questions, nothing else

Role:
- Think through your answer
"""
    }
]

In [38]:
test_queries = generate_data(bedrock_rt, model_id_sonnet, query_system_prompt, query_prompt, inference_config)
print(test_queries)

1. What are some healthy recipes that include quinoa?
2. Which users have shared their running or training experiences?
3. Are there any posts discussing mindfulness techniques for busy parents?
4. What gluten-free and dairy-free dessert recipes have been shared?
5. Are there any yoga or meditation posts that focus on flexibility and strength?


Let's select one of these generated questions. Copy and paste a generate question into the below cell. 

In [43]:
user ="What gluten-free and dairy-free dessert recipes have been shared?"

In [46]:
# Generate responses and measure runtime with Haiku model
start_time = time.time()
response1 = generate_data(bedrock_rt, model_id_sonnet, system_prompt_query_generation, user, inference_config)
print(response1)
output = query_oss(response1)
print("Sonnet took", time.time() - start_time, "to run")
print("Response after running query against Opensearch")
print(output)

<json>
{
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "post_type": "recipe"
          }
        },
        {
          "match": {
            "category": "food"
          }
        },
        {
          "match": {
            "tags": "gluten-free"
          }
        },
        {
          "match": {
            "tags": "dairy-free"
          }
        }
      ]
    }
  }
}
</json>
Sonnet took 4.676537752151489 to run
Response after running query against Opensearch
{'took': 47, 'timed_out': False, '_shards': {'total': 0, 'successful': 0, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 5.2397428, 'hits': [{'_index': 'userprofile_index', '_id': '9O-9m5EBiT38F8xt6mpC', '_score': 5.2397428, '_source': {'user_id': 'user_4', 'username': 'GlutenFreeFoodie', 'post_type': 'recipe', 'title': 'Gluten-Free Chocolate Chip Cookies (Dairy-Free Option)', 'content': "Craving something sweet but need a gluten-free and d

Above we can see that the query generated is accurate syntax to query data within Opensearch Collection.

---
## Clean Up

After we are done, delete the indexes for the collection.

In [ ]:
def delete_opensearch_serverless_indices(collection_id, client):
    # Create a boto3 client for OpenSearch Serverles
    try:
        client.indices.delete(index='_all')
    except Exception as e:
        print(f"An error occurred: {e}")
        
delete_opensearch_serverless_indices(collection_id, oss_client)

---
## Conclusion

We observed through this notebook how to create synthetic data to improve pace of testing with schemas and an approach to text-to-queryDSL using Claude 3 models.

This notebook allows you to change the model_id to Haiku, Sonnet and Sonnet 3.5 to test accuracy and latency between all three models.